In [1]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor 

Importing the training dataset

In [2]:
train = pd.read_csv('E:/Arvind/Kaggle/New York City Taxi Trip Duration/Input/train.csv')

In [3]:
train.shape

(1458644, 11)

In [4]:
train.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


In [5]:
print(train.describe)

<bound method NDFrame.describe of                 id  vendor_id      pickup_datetime     dropoff_datetime  \
0        id2875421          2  2016-03-14 17:24:55  2016-03-14 17:32:30   
1        id2377394          1  2016-06-12 00:43:35  2016-06-12 00:54:38   
2        id3858529          2  2016-01-19 11:35:24  2016-01-19 12:10:48   
3        id3504673          2  2016-04-06 19:32:31  2016-04-06 19:39:40   
4        id2181028          2  2016-03-26 13:30:55  2016-03-26 13:38:10   
...            ...        ...                  ...                  ...   
1458639  id2376096          2  2016-04-08 13:31:04  2016-04-08 13:44:02   
1458640  id1049543          1  2016-01-10 07:35:15  2016-01-10 07:46:10   
1458641  id2304944          2  2016-04-22 06:57:41  2016-04-22 07:10:25   
1458642  id2714485          1  2016-01-05 15:56:26  2016-01-05 16:02:39   
1458643  id1209952          1  2016-04-05 14:44:25  2016-04-05 14:47:43   

         passenger_count  pickup_longitude  pickup_latitude  \
0 

Feature Engineering

1) Encoding vendor_id           

In [6]:
train = pd.concat((train,pd.get_dummies(train['vendor_id'],prefix = 'vendor_id' )),axis=1)

In [7]:
train.groupby('vendor_id')['trip_duration'].mean()

vendor_id
1     845.438244
2    1058.643164
Name: trip_duration, dtype: float64

2) Pickup hour, day and week and then encoding

In [5]:
train['pickup_datetime'] =  pd.to_datetime(train['pickup_datetime'])

train['pickup_hour'] = train['pickup_datetime'].dt.hour
train['pickup_day'] = train['pickup_datetime'].dt.day
train['pickup_week'] = train['pickup_datetime'].dt.week

<ipython-input-5-645caa8235e0>:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  train['pickup_week'] = train.pickup_datetime.dt.week


In [9]:
train.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,vendor_id_1,vendor_id_2,pickup_hour,pickup_day,pickup_week
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,0,1,17,14,11
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663,1,0,0,12,23
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124,0,1,11,19,3
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429,0,1,19,6,14
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435,0,1,13,26,12


In [12]:
print(train.groupby('pickup_hour')['trip_duration'].mean())
print(train.groupby('pickup_day')['trip_duration'].mean())
print(train.groupby('pickup_week')['trip_duration'].mean())

pickup_hour
0      936.657302
1      903.087112
2      890.076934
3      890.070878
4      921.976444
5      822.298960
6      797.434943
7      831.758345
8      924.559199
9      933.528915
10     933.367605
11     966.430282
12     993.533079
13    1032.246359
14    1075.789264
15    1118.832533
16    1080.146285
17    1030.581619
18     981.583234
19     894.356679
20     879.461545
21     890.521221
22    1023.455921
23     925.091409
Name: trip_duration, dtype: float64
pickup_day
1      958.053978
2      928.860299
3      976.247221
4      947.549451
5     1020.272899
6      935.184170
7      900.626102
8      940.452475
9      945.018737
10     926.881474
11     934.831389
12     939.305731
13    1067.434301
14     972.608976
15     969.430961
16     952.495522
17     983.950988
18     947.133166
19     959.540262
20     949.521607
21     947.743779
22     963.256846
23     971.127793
24     989.043896
25     991.575221
26     985.632440
27     955.167025
28     968.629884
29   

In [13]:
print(train['pickup_hour'].value_counts().sort_index())
print(train['pickup_day'].value_counts().sort_index())
print(train['pickup_week'].value_counts().sort_index())

0     53248
1     38571
2     27972
3     20895
4     15792
5     15002
6     33248
7     55600
8     67053
9     67663
10    65437
11    68476
12    71873
13    71473
14    74292
15    71811
16    64313
17    76483
18    90600
19    90308
20    84072
21    84185
22    80492
23    69785
Name: pickup_hour, dtype: int64
1     46612
2     47752
3     47945
4     49655
5     50176
6     49476
7     48671
8     48248
9     49634
10    48867
11    48864
12    50080
13    49293
14    50488
15    49791
16    51027
17    48968
18    47925
19    49265
20    49198
21    48306
22    47254
23    41450
24    43876
25    46450
26    47595
27    46957
28    45891
29    46807
30    39135
31    22988
Name: pickup_day, dtype: int64
1     53204
2     56934
3     45309
4     54233
5     56163
6     59047
7     57158
8     58530
9     59679
10    57817
11    59200
12    55501
13    57443
14    60807
15    60060
16    55824
17    56869
18    59520
19    57598
20    58420
21    52284
22    53783
23    55259
2

Note: Training data has only till 26 week and then 53rd week, so we will only use hour and day in our feature list
Encoding them below

In [18]:
train = pd.concat((train,pd.get_dummies(train['pickup_hour'],prefix = 'pickup_hour' )),axis=1)
train = pd.concat((train,pd.get_dummies(train['pickup_day'],prefix = 'pickup_day' )),axis=1)

In [19]:
train.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,...,pickup_day_22,pickup_day_23,pickup_day_24,pickup_day_25,pickup_day_26,pickup_day_27,pickup_day_28,pickup_day_29,pickup_day_30,pickup_day_31
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,...,0,0,0,0,0,0,0,0,0,0
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,...,0,0,0,0,0,0,0,0,0,0
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,...,0,0,0,0,0,0,0,0,0,0
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,...,0,0,0,0,0,0,0,0,0,0
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,...,0,0,0,0,1,0,0,0,0,0


3) Distance using latitude and longitude

In [14]:
# Function that calculates distance between 4 np and returns np

def distance(s_lat, s_lng, e_lat, e_lng):
    
    # approximate radius of earth in km
    R = 6373.0
    
    s_lat = s_lat*np.pi/180.0                      
    s_lng = np.deg2rad(s_lng)     
    e_lat = np.deg2rad(e_lat)                       
    e_lng = np.deg2rad(e_lng)  
    
    d = np.sin((e_lat - s_lat)/2)**2 + np.cos(s_lat)*np.cos(e_lat) * np.sin((e_lng - s_lng)/2)**2
    
    return 2 * R * np.arcsin(np.sqrt(d)) 

In [15]:
train['distance'] = distance(train['pickup_latitude'],train['pickup_longitude'],train['dropoff_latitude'],train['dropoff_longitude'])

In [16]:
print(train.groupby(pd.qcut(train['distance'],4))['trip_duration'].mean())

distance
(-0.001, 1.232]       468.338471
(1.232, 2.094]        690.616156
(2.094, 3.877]        964.122610
(3.877, 1241.298]    1714.891856
Name: trip_duration, dtype: float64


4) store_and_fwd_flag -
EDA and then encoding

In [17]:
print(train.groupby('store_and_fwd_flag')['trip_duration'].mean())

store_and_fwd_flag
N     958.819706
Y    1080.763331
Name: trip_duration, dtype: float64


In [20]:
train = pd.concat((train,pd.get_dummies(train['store_and_fwd_flag'],prefix = 'store_and_fwd_flag' )),axis=1)

In [21]:
train.columns

Index(['id', 'vendor_id', 'pickup_datetime', 'dropoff_datetime',
       'passenger_count', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'store_and_fwd_flag',
       'trip_duration', 'vendor_id_1', 'vendor_id_2', 'pickup_hour',
       'pickup_day', 'pickup_week', 'distance', 'pickup_hour_0',
       'pickup_hour_1', 'pickup_hour_2', 'pickup_hour_3', 'pickup_hour_4',
       'pickup_hour_5', 'pickup_hour_6', 'pickup_hour_7', 'pickup_hour_8',
       'pickup_hour_9', 'pickup_hour_10', 'pickup_hour_11', 'pickup_hour_12',
       'pickup_hour_13', 'pickup_hour_14', 'pickup_hour_15', 'pickup_hour_16',
       'pickup_hour_17', 'pickup_hour_18', 'pickup_hour_19', 'pickup_hour_20',
       'pickup_hour_21', 'pickup_hour_22', 'pickup_hour_23', 'pickup_day_1',
       'pickup_day_2', 'pickup_day_3', 'pickup_day_4', 'pickup_day_5',
       'pickup_day_6', 'pickup_day_7', 'pickup_day_8', 'pickup_day_9',
       'pickup_day_10', 'pickup_day_11', 'pickup_day_12', 'pi

Test data import

In [42]:
test = pd.read_csv('E:/Arvind/Kaggle/New York City Taxi Trip Duration/Input/test.csv')

In [43]:
test.shape

(625134, 9)

In [44]:
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 625134 entries, 0 to 625133
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  625134 non-null  object 
 1   vendor_id           625134 non-null  int64  
 2   pickup_datetime     625134 non-null  object 
 3   passenger_count     625134 non-null  int64  
 4   pickup_longitude    625134 non-null  float64
 5   pickup_latitude     625134 non-null  float64
 6   dropoff_longitude   625134 non-null  float64
 7   dropoff_latitude    625134 non-null  float64
 8   store_and_fwd_flag  625134 non-null  object 
dtypes: float64(4), int64(2), object(3)
memory usage: 42.9+ MB
None


Feature Engineering

1) Encoding vendor_id           

In [45]:
test = pd.concat((test,pd.get_dummies(test['vendor_id'],prefix = 'vendor_id' )),axis=1)

2) Pickup hour, day

In [46]:
test['pickup_datetime'] = pd.to_datetime(test.pickup_datetime)

test['pickup_hour'] = test.pickup_datetime.dt.hour.astype(int)
test['pickup_day'] = test.pickup_datetime.dt.day.astype(int)


In [47]:
test['pickup_hour'] = test['pickup_hour'].astype(int)
test['pickup_day'] = test['pickup_day'].astype(int)


In [50]:
test = pd.concat((test,pd.get_dummies(test['pickup_hour'],prefix = 'pickup_hour' )),axis=1)
test = pd.concat((test,pd.get_dummies(test['pickup_day'],prefix = 'pickup_day' )),axis=1)

In [51]:
test.columns

Index(['id', 'vendor_id', 'pickup_datetime', 'passenger_count',
       'pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
       'dropoff_latitude', 'store_and_fwd_flag', 'vendor_id_1', 'vendor_id_2',
       'pickup_hour', 'pickup_day', 'pickup_hour_0', 'pickup_hour_1',
       'pickup_hour_2', 'pickup_hour_3', 'pickup_hour_4', 'pickup_hour_5',
       'pickup_hour_6', 'pickup_hour_7', 'pickup_hour_8', 'pickup_hour_9',
       'pickup_hour_10', 'pickup_hour_11', 'pickup_hour_12', 'pickup_hour_13',
       'pickup_hour_14', 'pickup_hour_15', 'pickup_hour_16', 'pickup_hour_17',
       'pickup_hour_18', 'pickup_hour_19', 'pickup_hour_20', 'pickup_hour_21',
       'pickup_hour_22', 'pickup_hour_23', 'pickup_day_1', 'pickup_day_2',
       'pickup_day_3', 'pickup_day_4', 'pickup_day_5', 'pickup_day_6',
       'pickup_day_7', 'pickup_day_8', 'pickup_day_9', 'pickup_day_10',
       'pickup_day_11', 'pickup_day_12', 'pickup_day_13', 'pickup_day_14',
       'pickup_day_15', 'pickup_day_16',

3) Distance using latitude and longitude

In [52]:
test['distance'] = distance(test['pickup_latitude'],test['pickup_longitude'],test['dropoff_latitude'],test['dropoff_longitude'])

4) store_and_fwd_flag -
Encoding

In [53]:
test = pd.concat((test,pd.get_dummies(test['store_and_fwd_flag'],prefix = 'store_and_fwd_flag' )),axis=1)

In [54]:
test.head()

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,vendor_id_1,...,pickup_day_25,pickup_day_26,pickup_day_27,pickup_day_28,pickup_day_29,pickup_day_30,pickup_day_31,distance,store_and_fwd_flag_N,store_and_fwd_flag_Y
0,id3004672,1,2016-06-30 23:59:58,1,-73.988129,40.732029,-73.990173,40.756680,N,1,...,0,0,0,0,0,1,0,2.747288,1,0
1,id3505355,1,2016-06-30 23:59:53,1,-73.964203,40.679993,-73.959808,40.655403,N,1,...,0,0,0,0,0,1,0,2.760105,1,0
2,id1217141,1,2016-06-30 23:59:47,1,-73.997437,40.737583,-73.986160,40.729523,N,1,...,0,0,0,0,0,1,0,1.306565,1,0
3,id2150126,2,2016-06-30 23:59:41,1,-73.956070,40.771900,-73.986427,40.730469,N,0,...,0,0,0,0,0,1,0,5.270742,1,0
4,id1598245,1,2016-06-30 23:59:33,1,-73.970215,40.761475,-73.961510,40.755890,N,1,...,0,0,0,0,0,1,0,0.961143,1,0


Fit RF Regressor with Grid Search

In [57]:
RF = RandomForestRegressor()

In [63]:
RF.fit(train[['vendor_id_1','vendor_id_2',
       'pickup_hour_0', 'pickup_hour_1','pickup_hour_2', 'pickup_hour_3', 'pickup_hour_4','pickup_hour_5',
       'pickup_hour_6', 'pickup_hour_7', 'pickup_hour_8', 'pickup_hour_9',
       'pickup_hour_10', 'pickup_hour_11', 'pickup_hour_12', 'pickup_hour_13',
       'pickup_hour_14', 'pickup_hour_15', 'pickup_hour_16', 'pickup_hour_17',
       'pickup_hour_18', 'pickup_hour_19', 'pickup_hour_20', 'pickup_hour_21',
       'pickup_hour_22', 'pickup_hour_23',
       'pickup_day_1','pickup_day_2', 'pickup_day_3', 'pickup_day_4', 'pickup_day_5',
       'pickup_day_6', 'pickup_day_7', 'pickup_day_8', 'pickup_day_9',
       'pickup_day_10', 'pickup_day_11', 'pickup_day_12', 'pickup_day_13',
       'pickup_day_14', 'pickup_day_15', 'pickup_day_16', 'pickup_day_17',
       'pickup_day_18', 'pickup_day_19', 'pickup_day_20', 'pickup_day_21',
       'pickup_day_22', 'pickup_day_23', 'pickup_day_24', 'pickup_day_25',
       'pickup_day_26', 'pickup_day_27', 'pickup_day_28', 'pickup_day_29',
       'pickup_day_30', 'pickup_day_31', 'store_and_fwd_flag_N',
       'store_and_fwd_flag_Y','distance']],train['trip_duration'])

RandomForestRegressor()

In [64]:
submission = RF.predict(test[['vendor_id_1','vendor_id_2',
       'pickup_hour_0', 'pickup_hour_1','pickup_hour_2', 'pickup_hour_3', 'pickup_hour_4','pickup_hour_5',
       'pickup_hour_6', 'pickup_hour_7', 'pickup_hour_8', 'pickup_hour_9',
       'pickup_hour_10', 'pickup_hour_11', 'pickup_hour_12', 'pickup_hour_13',
       'pickup_hour_14', 'pickup_hour_15', 'pickup_hour_16', 'pickup_hour_17',
       'pickup_hour_18', 'pickup_hour_19', 'pickup_hour_20', 'pickup_hour_21',
       'pickup_hour_22', 'pickup_hour_23',
       'pickup_day_1','pickup_day_2', 'pickup_day_3', 'pickup_day_4', 'pickup_day_5',
       'pickup_day_6', 'pickup_day_7', 'pickup_day_8', 'pickup_day_9',
       'pickup_day_10', 'pickup_day_11', 'pickup_day_12', 'pickup_day_13',
       'pickup_day_14', 'pickup_day_15', 'pickup_day_16', 'pickup_day_17',
       'pickup_day_18', 'pickup_day_19', 'pickup_day_20', 'pickup_day_21',
       'pickup_day_22', 'pickup_day_23', 'pickup_day_24', 'pickup_day_25',
       'pickup_day_26', 'pickup_day_27', 'pickup_day_28', 'pickup_day_29',
       'pickup_day_30', 'pickup_day_31', 'store_and_fwd_flag_N',
       'store_and_fwd_flag_Y','distance']])

In [65]:
pd.DataFrame(submission).head()

,0
0,621.63
1,659.69
2,538.37
3,936.58
4,273.07


In [66]:
submission = pd.concat((test['id'],pd.DataFrame(submission,columns = ['trip_duration'])),axis =1)

In [67]:
submission.head()

,id,trip_duration
0,id3004672,621.63
1,id3505355,659.69
2,id1217141,538.37
3,id2150126,936.58
4,id1598245,273.07


In [68]:
submission.to_csv('E:/Arvind/Kaggle/New York City Taxi Trip Duration/Output/submission.csv')